In [253]:
from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
#from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
from pathlib import Path
import torch
import re
from torch import Tensor
from fastai.text import Tokenizer, Vocab
import pandas as pd
import collections
import os
import pdb
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.metrics import roc_curve, auc

from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder


from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
#from pytorch_pretrained_bert.optimization import BertAdam

## Import Early Stopping

In [254]:
# import EarlyStopping
from pytorchtools import EarlyStopping

In [255]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
import torch.nn as nn
import torch.nn.functional as F 

In [256]:
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear

In [257]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


#!/usr/bin/python


logger = logging.getLogger()

def setup_file_logger(log_file):
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s -   %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)

def log(message):
    #outputs to Jupyter console
    print('{} {}'.format(datetime.datetime.now(), message))
    #outputs to file
    logger.info(message)

setup_file_logger('out.log')

# for i in range(10000):
#     log('Doing hard work here i=' + str(i))
#     log('Taking a nap now...')
#     time.sleep(1000)

In [258]:
import pathlib
print(pathlib.Path.cwd())

/data/home/gicgpu/Know-Your-Intent


In [259]:
!ls /data/home/gicgpu/Know-Your-Intent

BertPretrain.ipynb	       out.log		semhash_pipeline.ipynb
comprehensive_semhash_test.py  plots		table
datasets		       __pycache__	train_test_split.py
gitignore		       pytorchtools.py	updated_semhash_pipeline.ipynb
hashembedding		       README.md
intent_classification.ipynb    results.csv


In [260]:
def load_label_list(trainset_path):
    df=pd.read_csv(trainset_path, header=None, sep='\t')
    return df[1].unique()
    
label_list=load_label_list('./datasets/KL/AskUbuntu/train.csv')
print(label_list)

['None' 'Make Update' 'Setup Printer' 'Software Recommendation' 'Shutdown Computer']


# BERT Paths

In [323]:
DATA_PATH=Path('/data/home/gicgpu/Know-Your-Intent/datasets/KL/AskUbuntu/')
#DATA_PATH.mkdir(exist_ok=True)

PATH=Path('/data/home/gicgpu/Know-Your-Intent/datasets/KL/AskUbuntu/')
# PATH.mkdir(exist_ok=True)

CLAS_DATA_PATH=PATH/'class/'
#CLAS_DATA_PATH.mkdir(exist_ok=True)

# model_state_dict = None

# BERT_PRETRAINED_PATH = Path('../trained_model/')
BERT_PRETRAINED_PATH = 'bert-base-uncased'
# BERT_PRETRAINED_PATH = Path('../../complaints/bert/pretrained-weights/cased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../complaints/bert/pretrained-weights/uncased_L-24_H-1024_A-16/')


# BERT_FINETUNED_WEIGHTS = Path('../trained_model/toxic_comments')

#PYTORCH_PRETRAINED_BERT_CACHE = BERT_PRETRAINED_PATH/'cache/'
#PYTORCH_PRETRAINED_BERT_CACHE.mkdir(exist_ok=True)

# output_model_file = os.path.join(BERT_FINETUNED_WEIGHTS, "pytorch_model.bin")

# Load a trained model that you have fine-tuned
# model_state_dict = torch.load('/data/home/gicgpu/sprint4/model/finetuned_on_scraped.bin') #(!!!) load finetuned model here
# model_state_dict = torch.load(os.path.join('/data/home/gicgpu/sprint4/model/', "bert27trainontrainthenVal.bin"))

In [324]:
args = {
    "train_size": -1,
    "val_size": -1,
    "full_data_dir": DATA_PATH,
    "data_dir": PATH,
    "task_name": "gic_bert",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": CLAS_DATA_PATH,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "learning_rate": 3e-5,
    "num_train_epochs": 20.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": False,
    "loss_scale": 128,
}

In [325]:
class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, config, num_labels=2):
        super(BertForMultiLabelSequenceClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = CrossEntropyLoss()

            loss = loss_fct(logits, labels.long())
            return loss
        else:
            return logits
        
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [326]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids

In [327]:
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
    
    def get_test_examples(self, data_dir, data_file_name, size=-1):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError() 

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

In [328]:
class MultiLabelTextProcessor(DataProcessor):
    
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.labels = None
    
    
    def get_train_examples(self, data_dir, size=-1):
        filename = 'train.csv'
#         filename = 'val.csv' ## (!!!) switch to train on val
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, filename)))
        if size == -1:
            data_df = pd.read_csv(os.path.join(data_dir, filename), header=None, sep='\t')
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df, "train")
        else:
            data_df = pd.read_csv(os.path.join(data_dir, filename), header=None, sep='\t')
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df.sample(size), "train")
        
    def get_dev_examples(self, data_dir, size=-1):
        """See base class."""
        filename = 'test.csv'
        if size == -1:
            data_df = pd.read_csv(os.path.join(data_dir, filename), header=None, sep='\t')
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df, "dev")
        else:
            data_df = pd.read_csv(os.path.join(data_dir, filename), header=None, sep='\t')
#             data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
            return self._create_examples(data_df.sample(size), "dev")
    
    def get_test_examples(self, data_dir, data_file_name, size=-1):
        data_df = pd.read_csv(os.path.join(data_dir, data_file_name), header=None, sep='\t')
#         data_df['comment_text'] = data_df['comment_text'].apply(cleanHtml)
        if size == -1:
            return self._create_examples(data_df, "test")
        else:
            return self._create_examples(data_df.sample(size), "test")

    def get_labels(self):
        """See base class."""
        if self.labels == None:
            df=pd.read_csv(os.path.join(data_dir, filename), header=None, sep='\t')
            self.labels = df[1].unique()
#             self.labels = list(pd.read_csv(os.path.join(self.data_dir, "classes.txt"),header=None)[0].values)
        return self.labels

    def _create_examples(self, df, set_type, labels_available=True):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, row) in enumerate(df.values):
            guid = i # (!!!) change if any changes to dataframe
            text_a = row[0] # (!!!) change if any changes to dataframe
            if labels_available:
                labels = row[1] # (!!!) change if any changes to dataframe
            else:
                labels = []
            examples.append(
                InputExample(guid=guid, text_a=text_a, labels=labels))
        return examples

In [329]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}
#     print(label_map)
    features = []
    for (ex_index, example) in enumerate(examples):
#         try:
        tokens_a = tokenizer.tokenize(example.text_a)
#             print("Success!")
#         except Exception as e:
#             print("Failed index:",ex_index)
#             print("Failed ID:",example.guid)
#             print("Failed Text:",example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
#         print('exxxample', example.labels)
        labels_ids = label_map[example.labels]
#         for label in example.labels:
#             print(label)
#             labels_ids.append(float(label_map[label]))

#         label_id = label_map[example.label]
        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [330]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [419]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    print(outputs.cpu() == labels.cpu().long())
    return torch.sum(outputs.cpu() == labels.cpu().long())

# def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
#     "Compute accuracy when `y_pred` and `y_true` are the same size."
#     if sigmoid: y_pred = y_pred.sigmoid()
#     print('dfdfdfdf', np.argmax(y_pred, axis=1), y_true, np.argmax(y_pred, axis=1)==)
#     return (np.argmax(y_pred.cpu())==y_true.cpu().long).mean().item()
# #     return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

In [382]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

In [383]:
processors = {
    "gic_bert": MultiLabelTextProcessor
} # use default multiclass here instead lik

# Setup GPU parameters

if args["local_rank"] == -1 or args["no_cuda"]:
    device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")
    n_gpu = torch.cuda.device_count()
#     n_gpu = 1
else:
    torch.cuda.set_device(args['local_rank'])
    device = torch.device("cuda", args['local_rank'])
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args['local_rank'] != -1), args['fp16']))

04/24/2019 11:30:23 - INFO - root -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [384]:
args['train_batch_size'] = int(args['train_batch_size'] / args['gradient_accumulation_steps'])


In [385]:
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
if n_gpu > 0:
    torch.cuda.manual_seed_all(args['seed'])

In [386]:
task_name = args['task_name'].lower()

if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))

In [387]:
processor = processors[task_name](args['data_dir'])
label_list = load_label_list('./datasets/KL/AskUbuntu/train.csv')
num_labels = len(label_list)

In [388]:
label_list


array(['None', 'Make Update', 'Setup Printer', 'Software Recommendation', 'Shutdown Computer'], dtype=object)

In [389]:
tokenizer = BertTokenizer.from_pretrained(args['bert_model'], do_lower_case=args['do_lower_case'])


04/24/2019 11:30:25 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/gicgpu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [390]:
train_examples = None
num_train_steps = None
if args['do_train']:
    train_examples = processor.get_train_examples(args['full_data_dir'], size=args['train_size'])
#     train_examples = processor.get_train_examples(args['data_dir'], size=args['train_size'])
    num_train_steps = int(
        len(train_examples) / args['train_batch_size'] / args['gradient_accumulation_steps'] * args['num_train_epochs'])

04/24/2019 11:30:25 - INFO - root -   LOOKING AT /data/home/gicgpu/Know-Your-Intent/datasets/KL/AskUbuntu/train.csv


In [391]:
# Prepare model
def get_model():
#     pdb.set_trace()
#     if model_state_dict:
#         model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels, state_dict=model_state_dict)
#     else:
    model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels)
    return model

model = get_model()

if args['fp16']:
    model.half()
model.to(device)
if args['local_rank'] != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    model = DDP(model)
elif n_gpu > 1:
    model = torch.nn.DataParallel(model)

04/24/2019 11:30:26 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/gicgpu/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/24/2019 11:30:26 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/gicgpu/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpigtnf8lf
04/24/2019 11:30:30 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,

In [392]:
from torch.optim.lr_scheduler import _LRScheduler, Optimizer

class CyclicLR(object):
    """Sets the learning rate of each parameter group according to
    cyclical learning rate policy (CLR). The policy cycles the learning
    rate between two boundaries with a constant frequency, as detailed in
    the paper `Cyclical Learning Rates for Training Neural Networks`_.
    The distance between the two boundaries can be scaled on a per-iteration
    or per-cycle basis.
    Cyclical learning rate policy changes the learning rate after every batch.
    `batch_step` should be called after a batch has been used for training.
    To resume training, save `last_batch_iteration` and use it to instantiate `CycleLR`.
    This class has three built-in policies, as put forth in the paper:
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    This implementation was adapted from the github repo: `bckenstler/CLR`_
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        base_lr (float or list): Initial learning rate which is the
            lower boundary in the cycle for eachparam groups.
            Default: 0.001
        max_lr (float or list): Upper boundaries in the cycle for
            each parameter group. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function. Default: 0.006
        step_size (int): Number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch. Default: 2000
        mode (str): One of {triangular, triangular2, exp_range}.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
            Default: 'triangular'
        gamma (float): Constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
            Default: 1.0
        scale_fn (function): Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
            Default: None
        scale_mode (str): {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle).
            Default: 'cycle'
        last_batch_iteration (int): The index of the last batch. Default: -1
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> scheduler = torch.optim.CyclicLR(optimizer)
        >>> data_loader = torch.utils.data.DataLoader(...)
        >>> for epoch in range(10):
        >>>     for batch in data_loader:
        >>>         scheduler.batch_step()
        >>>         train_batch(...)
    .. _Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
    .. _bckenstler/CLR: https://github.com/bckenstler/CLR
    """

    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

#         if not isinstance(optimizer, Optimizer):
#             raise TypeError('{} is not an Optimizer'.format(
#                 type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [393]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps
if args['local_rank'] != -1:
    t_total = t_total // torch.distributed.get_world_size()
if args['fp16']:
    try:
        from apex.optimizers import FP16_Optimizer
        from apex.optimizers import FusedAdam
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

    optimizer = FusedAdam(optimizer_grouped_parameters,
                          lr=args['learning_rate'],
                          bias_correction=False,
                          max_grad_norm=1.0)
    if args['loss_scale'] == 0:
        optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    else:
        optimizer = FP16_Optimizer(optimizer, static_loss_scale=args['loss_scale'])

else:
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=args['learning_rate'],
                         warmup=args['warmup_proportion'],
                         t_total=t_total)

scheduler = CyclicLR(optimizer, base_lr=2e-5, max_lr=5e-5, step_size=2500, last_batch_iteration=0)

In [450]:
# Eval Fn
eval_examples = processor.get_dev_examples(args['data_dir'], size=args['val_size'])
def eval():
    args['output_dir'].mkdir(exist_ok=True)

    
    eval_features = convert_examples_to_features(
        eval_examples, label_list, args['max_seq_length'], tokenizer)
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in eval_features], dtype=torch.float)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args['eval_batch_size'])
    
    all_logits = None
    all_labels = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():
            tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

#         logits = logits.detach().cpu().numpy()
#         label_ids = label_ids.to('cpu').numpy()
#         tmp_eval_accuracy = accuracy(logits, label_ids)
        tmp_eval_accuracy = accuracy(logits, label_ids)
        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        if all_labels is None:
            all_labels = label_ids.detach().cpu().numpy()
        else:    
            all_labels = np.concatenate((all_labels, label_ids.detach().cpu().numpy()), axis=0)
        

        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy

        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    
#     ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
#     for i in range(num_labels):
#         fpr[i], tpr[i], _ = roc_curve(all_labels[:, i], all_logits[:, i])
#         roc_auc[i] = auc(fpr[i], tpr[i])
        
#     # Compute micro-average ROC curve and ROC area
#     fpr["micro"], tpr["micro"], _ = roc_curve(all_labels.ravel(), all_logits.ravel())
#     roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])



#     F1 calcualation
    # Compute F1 micro
    y_true = pd.read_csv("/data/home/gicgpu/Know-Your-Intent/datasets/KL/AskUbuntu/test.csv", header=None, sep='\t')
#     y_true_results = y_true[['ATAD', 'CFC', 'Corporate Tax',
#        'Country-by-Country Reporting', 'DAC6', 'Digital Tax',
#        'Double Tax Treaty', 'Economic substance', 'GAAR',
#        'Goods and services tax (GST)', 'Interest deductibility', 'Local file',
#        'MLI', 'Management of a Tax Function', 'Mandatory disclosure rules',
#        'Master file', 'Mutual agreement procedure', 'Permanent Establishment',
#        'Principal purpose test', 'Section 892', 'Sovereign Immunity',
#        'Tax Governance Framework', 'Tax audit', 'Tax compliance',
#        'Tax dispute', 'Value-added tax (VAT)', 'Withholding Tax']]
    
    y_true_results = y_true[1]
    pred_results = pd.DataFrame(all_logits, columns=label_list)
    pred_results = pred_results.idxmax(axis=1)
#     print(y_true_results)
    f1_micro = f1_score(y_true_results, pred_results, average='micro')

    
    result = {'eval_loss': eval_loss,
              'eval_accuracy': eval_accuracy,
#               'loss': tr_loss/nb_tr_steps,
              'roc_auc': roc_auc,
             'weighted f1_score': f1_micro}
    
    output_eval_file = os.path.join(args['output_dir'], "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
#             writer.write("%s = %s\n" % (key, str(result[key])))
    return result

# Load training data


In [423]:
train_features = convert_examples_to_features(
    train_examples, label_list, args['max_seq_length'], tokenizer)

In [424]:
# for this in train_features:
#     print(this.label_ids)

In [425]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args['train_batch_size'])
logger.info("  Num steps = %d", num_train_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in train_features], dtype=torch.float)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
if args['local_rank'] == -1:
    train_sampler = RandomSampler(train_data)
else:
    train_sampler = DistributedSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args['train_batch_size'])

04/24/2019 11:38:05 - INFO - root -   ***** Running training *****
04/24/2019 11:38:05 - INFO - root -     Num examples = 53
04/24/2019 11:38:05 - INFO - root -     Batch size = 8
04/24/2019 11:38:05 - INFO - root -     Num steps = 132


In [426]:
from tqdm import tqdm_notebook as tqdm


# Train Model

In [427]:
def fit(num_epocs=args['num_train_epochs']):
    global_step = 0
    model.train()
    
    for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                optimizer.backward(loss)
            else:
                loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
    #             scheduler.batch_step()
                # modify learning rate with special warm up BERT uses
                lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_this_step
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

        logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
        logger.info('Eval after epoc {}'.format(i_+1))
        eval()

In [428]:
def fit_with_early_stopping(num_epocs=args['num_train_epochs'], patience=4):
    early_stopping = EarlyStopping(patience=patience, verbose=True)
#     train_losses = []
#     valid_losses = []
#     avg_train_losses = []
#     avg_valid_losses = [] 
    
    global_step = 0
    
    model.train()
    
    for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                optimizer.backward(loss)
            else:
                loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
    #             scheduler.batch_step()
                # modify learning rate with special warm up BERT uses
                lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_this_step
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

        logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
        logger.info('Eval after epoc {}'.format(i_+1))
        eval_results = eval()
        # instead of early stopping on val loss we want to stop if f1 stops improving
        val_f1 = eval_results['weighted f1_score']
        early_stopping(val_f1, model)
        if early_stopping.early_stop:
            print("Stopping early")
            break
        

        


In [429]:
torch.cuda.is_available()

True

In [430]:
torch.backends.cudnn.enabled

True

# PAUSE Here, Don't train if loading pretrained

In [431]:
# Freeze BERT layers for 1 epoch
model.freeze_bert_encoder()

In [456]:
fit_with_early_stopping(1)

04/24/2019 11:54:56 - INFO - root -   Loss after epoc 0.08920026783432279
04/24/2019 11:54:56 - INFO - root -   Eval after epoc 1
04/24/2019 11:54:56 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:54:56 - INFO - root -     Num examples = 109
04/24/2019 11:54:56 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)


04/24/2019 11:55:00 - INFO - root -   ***** Eval results *****
04/24/2019 11:55:00 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:55:00 - INFO - root -     eval_loss = 0.391350830239909
04/24/2019 11:55:00 - INFO - root -     roc_auc = {}
04/24/2019 11:55:00 - INFO - root -     weighted f1_score = 0.908256880733945


tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
Validation loss decreased (inf --> 0.908257).  Saving model ...


In [457]:
fit(20)

04/24/2019 11:55:16 - INFO - root -   Loss after epoc 0.07830638651336942
04/24/2019 11:55:16 - INFO - root -   Eval after epoc 1
04/24/2019 11:55:16 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:55:16 - INFO - root -     Num examples = 109
04/24/2019 11:55:16 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)


04/24/2019 11:55:20 - INFO - root -   ***** Eval results *****
04/24/2019 11:55:20 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:55:20 - INFO - root -     eval_loss = 0.3975697412554707
04/24/2019 11:55:20 - INFO - root -     roc_auc = {}
04/24/2019 11:55:20 - INFO - root -     weighted f1_score = 0.908256880733945


tensor([1, 1, 1, 1, 1], dtype=torch.uint8)


04/24/2019 11:55:23 - INFO - root -   Loss after epoc 0.06230604116405759
04/24/2019 11:55:23 - INFO - root -   Eval after epoc 2
04/24/2019 11:55:23 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:55:23 - INFO - root -     Num examples = 109
04/24/2019 11:55:23 - INFO - root -     Batch size = 8


...

04/24/2019 11:57:08 - INFO - root -   ***** Eval results *****
04/24/2019 11:57:08 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:57:08 - INFO - root -     eval_loss = 0.42897934592994197
04/24/2019 11:57:08 - INFO - root -     roc_auc = {}
04/24/2019 11:57:08 - INFO - root -     weighted f1_score = 0.908256880733945


tensor([1, 1, 1, 1, 1], dtype=torch.uint8)


04/24/2019 11:57:11 - INFO - root -   Loss after epoc 0.020254544647676603
04/24/2019 11:57:11 - INFO - root -   Eval after epoc 16
04/24/2019 11:57:11 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:57:11 - INFO - root -     Num examples = 109
04/24/2019 11:57:11 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)


KeyboardInterrupt: 

In [453]:
eval()

04/24/2019 11:51:35 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:51:35 - INFO - root -     Num examples = 109
04/24/2019 11:51:35 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)


04/24/2019 11:51:39 - INFO - root -   ***** Eval results *****
04/24/2019 11:51:39 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:51:39 - INFO - root -     eval_loss = 1.5185668979372298
04/24/2019 11:51:39 - INFO - root -     roc_auc = {}
04/24/2019 11:51:39 - INFO - root -     weighted f1_score = 0.3394495412844037


tensor([0, 0, 0, 0, 0], dtype=torch.uint8)


{'eval_loss': 1.5185668979372298,
 'eval_accuracy': tensor(0),
 'roc_auc': {},
 'weighted f1_score': 0.3394495412844037}

In [454]:
model.unfreeze_bert_encoder()


In [455]:
fit_with_early_stopping(20)

04/24/2019 11:51:46 - INFO - root -   Loss after epoc 1.5162230219159807
04/24/2019 11:51:46 - INFO - root -   Eval after epoc 1
04/24/2019 11:51:46 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:51:46 - INFO - root -     Num examples = 109
04/24/2019 11:51:46 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 1, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 1, 0, 0, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 0, 0, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)


04/24/2019 11:51:50 - INFO - root -   ***** Eval results *****
04/24/2019 11:51:50 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:51:50 - INFO - root -     eval_loss = 1.4198031084878104
04/24/2019 11:51:50 - INFO - root -     roc_auc = {}
04/24/2019 11:51:50 - INFO - root -     weighted f1_score = 0.5596330275229358


tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
Validation loss decreased (inf --> 0.559633).  Saving model ...


04/24/2019 11:51:56 - INFO - root -   Loss after epoc 1.329736522265843
04/24/2019 11:51:56 - INFO - root -   Eval after epoc 2
04/24/2019 11:51:56 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:51:56 - INFO - root -     Num examples = 109
04/24/2019 11:51:56 - INFO - root -     Batch size = 8


...

tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
EarlyStopping counter: 3 out of 4


04/24/2019 11:52:19 - INFO - root -   Loss after epoc 0.13255969594631875
04/24/2019 11:52:19 - INFO - root -   Eval after epoc 5
04/24/2019 11:52:19 - INFO - root -   ***** Running evaluation *****
04/24/2019 11:52:19 - INFO - root -     Num examples = 109
04/24/2019 11:52:19 - INFO - root -     Batch size = 8


tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)


04/24/2019 11:52:24 - INFO - root -   ***** Eval results *****
04/24/2019 11:52:24 - INFO - root -     eval_accuracy = tensor(0)
04/24/2019 11:52:24 - INFO - root -     eval_loss = 0.3901435684944902
04/24/2019 11:52:24 - INFO - root -     roc_auc = {}
04/24/2019 11:52:24 - INFO - root -     weighted f1_score = 0.8990825688073395


tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
EarlyStopping counter: 4 out of 4
Stopping early


# Train on train set first, now train on val

In [62]:
train_examples = None
num_train_steps = None
if args['do_train']:
    train_examples = processor.get_dev_examples(args['full_data_dir'], size=args['train_size'])
#     train_examples = processor.get_train_examples(args['data_dir'], size=args['train_size'])
    num_train_steps = int(
        len(train_examples) / args['train_batch_size'] / args['gradient_accumulation_steps'] * args['num_train_epochs'])

In [63]:
train_features = convert_examples_to_features(
    train_examples, label_list, args['max_seq_length'], tokenizer)

In [64]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args['train_batch_size'])
logger.info("  Num steps = %d", num_train_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in train_features], dtype=torch.float)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
if args['local_rank'] == -1:
    train_sampler = RandomSampler(train_data)
else:
    train_sampler = DistributedSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args['train_batch_size'])

04/16/2019 03:48:15 - INFO - root -   ***** Running training *****
04/16/2019 03:48:15 - INFO - root -     Num examples = 198
04/16/2019 03:48:15 - INFO - root -     Batch size = 8
04/16/2019 03:48:15 - INFO - root -     Num steps = 495


In [65]:
fit_with_early_stopping(20)

04/16/2019 03:48:46 - INFO - root -   Loss after epoc 3.405038070678711
04/16/2019 03:48:46 - INFO - root -   Eval after epoc 1
04/16/2019 03:48:56 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:48:56 - INFO - root -     Num examples = 198
04/16/2019 03:48:56 - INFO - root -     Batch size = 8
04/16/2019 03:49:03 - INFO - root -   ***** Eval results *****
04/16/2019 03:49:03 - INFO - root -     eval_accuracy = 0.9139543615206324
04/16/2019 03:49:03 - INFO - root -     eval_loss = 3.4611298382282256
04/16/2019 03:49:03 - INFO - root -     roc_auc = {0: 0.6948391922213912, 1: 0.7912371134020618, 2: 0.531578947368421, 3: 0.6554659498207884, 4: 0.7497794766245223, 5: 0.799342105263158, 6: 0.722872340425532, 7: 0.993923611111111, 8: 0.4988780852655198, 9: 0.8033154121863799, 10: 0.9086021505376344, 11: 0.6963350785340314, 12: 0.9148936170212767, 13: 0.818342151675485, 14: 0.9207446808510638, 15: 0.7818100358422938, 16: 0.5489902767389678, 17: 0.758894442810938, 18: 0.9316239

Validation loss decreased (inf --> 0.361250).  Saving model ...


04/16/2019 03:49:17 - INFO - root -   Loss after epoc 3.393074116706848
04/16/2019 03:49:17 - INFO - root -   Eval after epoc 2
04/16/2019 03:49:26 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:49:26 - INFO - root -     Num examples = 198
04/16/2019 03:49:26 - INFO - root -     Batch size = 8
04/16/2019 03:49:34 - INFO - root -   ***** Eval results *****
04/16/2019 03:49:34 - INFO - root -     eval_accuracy = 0.9070333013630877
04/16/2019 03:49:34 - INFO - root -     eval_loss = 3.179743395447731
04/16/2019 03:49:34 - INFO - root -     roc_auc = {0: 0.700074794315632, 1: 0.7809278350515464, 2: 0.5375, 3: 0.6586021505376344, 4: 0.7547780064686856, 5: 0.8157894736842104, 6: 0.7308510638297873, 7: 0.9956597222222222, 8: 0.5026178010471204, 9: 0.8051075268817205, 10: 0.9130824372759856, 11: 0.7053103964098728, 12: 0.9297872340425533, 13: 0.823045267489712, 14: 0.9361702127659575, 15: 0.7889784946236559, 16: 0.5646970830216903, 17: 0.7630108791531902, 18: 0.9452991452991453

EarlyStopping counter: 1 out of 4


04/16/2019 03:49:46 - INFO - root -   Loss after epoc 3.03188125371933
04/16/2019 03:49:46 - INFO - root -   Eval after epoc 3
04/16/2019 03:49:55 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:49:55 - INFO - root -     Num examples = 198
04/16/2019 03:49:55 - INFO - root -     Batch size = 8
04/16/2019 03:50:03 - INFO - root -   ***** Eval results *****
04/16/2019 03:50:03 - INFO - root -     eval_accuracy = 0.8963711237666583
04/16/2019 03:50:03 - INFO - root -     eval_loss = 2.758124282360077
04/16/2019 03:50:03 - INFO - root -     roc_auc = {0: 0.7105459985041137, 1: 0.779639175257732, 2: 0.5546052631578947, 3: 0.6698028673835126, 4: 0.7591884739782417, 5: 0.8282894736842106, 6: 0.7457446808510638, 7: 0.9965277777777778, 8: 0.5071054599850411, 9: 0.8095878136200717, 10: 0.9238351254480287, 11: 0.7083021690351533, 12: 0.9569148936170213, 13: 0.8271604938271605, 14: 0.9627659574468085, 15: 0.7979390681003584, 16: 0.5863874345549739, 17: 0.7738900323434283, 18: 0.9521

EarlyStopping counter: 2 out of 4


04/16/2019 03:50:15 - INFO - root -   Loss after epoc 2.623803045749664
04/16/2019 03:50:15 - INFO - root -   Eval after epoc 4
04/16/2019 03:50:24 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:50:24 - INFO - root -     Num examples = 198
04/16/2019 03:50:24 - INFO - root -     Batch size = 8
04/16/2019 03:50:32 - INFO - root -   ***** Eval results *****
04/16/2019 03:50:32 - INFO - root -     eval_accuracy = 0.8782267161089965
04/16/2019 03:50:32 - INFO - root -     eval_loss = 2.282428666353226
04/16/2019 03:50:32 - INFO - root -     roc_auc = {0: 0.7210172026925953, 1: 0.7719072164948454, 2: 0.6328947368421053, 3: 0.689516129032258, 4: 0.7756542193472508, 5: 0.8651315789473684, 6: 0.7542553191489362, 7: 0.9982638888888888, 8: 0.5534779356768885, 9: 0.8131720430107526, 10: 0.9278673835125448, 11: 0.7217651458489155, 12: 0.9659574468085106, 13: 0.8330393885949442, 14: 0.972872340425532, 15: 0.818100358422939, 16: 0.6387434554973822, 17: 0.7874154660394, 18: 0.95555555

EarlyStopping counter: 3 out of 4


04/16/2019 03:50:45 - INFO - root -   Loss after epoc 2.1838810455799105
04/16/2019 03:50:45 - INFO - root -   Eval after epoc 5
04/16/2019 03:50:54 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:50:54 - INFO - root -     Num examples = 198
04/16/2019 03:50:54 - INFO - root -     Batch size = 8
04/16/2019 03:51:01 - INFO - root -   ***** Eval results *****
04/16/2019 03:51:01 - INFO - root -     eval_accuracy = 0.8576505810323388
04/16/2019 03:51:01 - INFO - root -     eval_loss = 1.8611076545715333
04/16/2019 03:51:01 - INFO - root -     roc_auc = {0: 0.7329842931937173, 1: 0.7603092783505155, 2: 0.7118421052631578, 3: 0.7155017921146953, 4: 0.804175242575713, 5: 0.9072368421052631, 6: 0.7718085106382978, 7: 1.0, 8: 0.6110695587135377, 9: 0.8118279569892473, 10: 0.935483870967742, 11: 0.7292445774121167, 12: 0.9797872340425533, 13: 0.8430335097001764, 14: 0.9845744680851064, 15: 0.8561827956989249, 16: 0.6985789080029917, 17: 0.8021170244045869, 18: 0.9555555555555555,

EarlyStopping counter: 4 out of 4
Stopping early


In [66]:
fit(5)

04/16/2019 03:52:35 - INFO - root -   Loss after epoc 1.8854332876205444
04/16/2019 03:52:35 - INFO - root -   Eval after epoc 1
04/16/2019 03:52:44 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:52:44 - INFO - root -     Num examples = 198
04/16/2019 03:52:44 - INFO - root -     Batch size = 8
04/16/2019 03:52:52 - INFO - root -   ***** Eval results *****
04/16/2019 03:52:52 - INFO - root -     eval_accuracy = 0.8557800187004937
04/16/2019 03:52:52 - INFO - root -     eval_loss = 1.8312656980752946
04/16/2019 03:52:52 - INFO - root -     roc_auc = {0: 0.7359760658189978, 1: 0.7603092783505155, 2: 0.7171052631578947, 3: 0.7172939068100358, 4: 0.805351367244928, 5: 0.9072368421052631, 6: 0.7712765957446809, 7: 1.0, 8: 0.6155572176514585, 9: 0.8122759856630825, 10: 0.9345878136200717, 11: 0.7314884068810771, 12: 0.9803191489361702, 13: 0.8447971781305115, 14: 0.9851063829787234, 15: 0.8615591397849462, 16: 0.706058339566193, 17: 0.8027050867391944, 18: 0.9555555555555555,

04/16/2019 03:53:04 - INFO - root -   Loss after epoc 1.8100076556205749
04/16/2019 03:53:04 - INFO - root -   Eval after epoc 2
04/16/2019 03:53:14 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:53:14 - INFO - root -     Num examples = 198
04/16/2019 03:53:14 - INFO - root -     Batch size = 8
04/16/2019 03:53:21 - INFO - root -   ***** Eval results *****
04/16/2019 03:53:21 - INFO - root -     eval_accuracy = 0.8490460256133416
04/16/2019 03:53:21 - INFO - root -     eval_loss = 1.7343215250968933
04/16/2019 03:53:21 - INFO - root -     roc_auc = {0: 0.7389678384442783, 1: 0.7590206185567011, 2: 0.7480263157894737, 3: 0.7262544802867383, 4: 0.8097618347544838, 5: 0.9098684210526315, 6: 0.773404255319149, 7: 1.0, 8: 0.6327599102468212, 9: 0.8136200716845878, 10: 0.9368279569892474, 11: 0.7344801795063575, 12: 0.9797872340425533, 13: 0.8506760728982952, 14: 0.9851063829787234, 15: 0.8705197132616487, 16: 0.7329842931937173, 17: 0.8065274919141429, 18: 0.9555555555555555

04/16/2019 03:53:34 - INFO - root -   Loss after epoc 1.7063525021076202
04/16/2019 03:53:34 - INFO - root -   Eval after epoc 3
04/16/2019 03:53:43 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:53:43 - INFO - root -     Num examples = 198
04/16/2019 03:53:43 - INFO - root -     Batch size = 8
04/16/2019 03:53:50 - INFO - root -   ***** Eval results *****
04/16/2019 03:53:50 - INFO - root -     eval_accuracy = 0.8426861233181424
04/16/2019 03:53:50 - INFO - root -     eval_loss = 1.5936561942100524
04/16/2019 03:53:50 - INFO - root -     roc_auc = {0: 0.737471952131638, 1: 0.7590206185567011, 2: 0.7868421052631579, 3: 0.7401433691756272, 4: 0.8247574242869744, 5: 0.9151315789473684, 6: 0.7819148936170213, 7: 1.0, 8: 0.6604338070306657, 9: 0.8167562724014338, 10: 0.9390681003584229, 11: 0.7464472700074795, 12: 0.9803191489361702, 13: 0.8612580834803057, 14: 0.9851063829787234, 15: 0.882168458781362, 16: 0.7711293941660433, 17: 0.8132902087621288, 18: 0.9572649572649573,

04/16/2019 03:54:03 - INFO - root -   Loss after epoc 1.5536206257343292
04/16/2019 03:54:03 - INFO - root -   Eval after epoc 4
04/16/2019 03:54:12 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:54:12 - INFO - root -     Num examples = 198
04/16/2019 03:54:12 - INFO - root -     Batch size = 8
04/16/2019 03:54:20 - INFO - root -   ***** Eval results *****
04/16/2019 03:54:20 - INFO - root -     eval_accuracy = 0.837448568055124
04/16/2019 03:54:20 - INFO - root -     eval_loss = 1.432760897874832
04/16/2019 03:54:20 - INFO - root -     roc_auc = {0: 0.7419596110695588, 1: 0.7564432989690721, 2: 0.819078947368421, 3: 0.7656810035842294, 4: 0.8391649514848574, 5: 0.9223684210526315, 6: 0.7877659574468086, 7: 1.0, 8: 0.6731488406881078, 9: 0.8216845878136201, 10: 0.9439964157706093, 11: 0.7636499626028421, 12: 0.9808510638297873, 13: 0.884185773074662, 14: 0.9845744680851064, 15: 0.8938172043010753, 16: 0.8167539267015707, 17: 0.8191708321082034, 18: 0.9606837606837606, 1

04/16/2019 03:54:32 - INFO - root -   Loss after epoc 1.3964218974113465
04/16/2019 03:54:32 - INFO - root -   Eval after epoc 5
04/16/2019 03:54:41 - INFO - root -   ***** Running evaluation *****
04/16/2019 03:54:41 - INFO - root -     Num examples = 198
04/16/2019 03:54:41 - INFO - root -     Batch size = 8
04/16/2019 03:54:49 - INFO - root -   ***** Eval results *****
04/16/2019 03:54:49 - INFO - root -     eval_accuracy = 0.8413767477478644
04/16/2019 03:54:49 - INFO - root -     eval_loss = 1.2948837733268739
04/16/2019 03:54:49 - INFO - root -     roc_auc = {0: 0.7471952131637996, 1: 0.7564432989690723, 2: 0.8368421052631579, 3: 0.7948028673835125, 4: 0.8573948838576889, 5: 0.9322368421052631, 6: 0.800531914893617, 7: 1.0, 8: 0.6970830216903515, 9: 0.8293010752688172, 10: 0.9493727598566308, 11: 0.7733732236350037, 12: 0.9819148936170212, 13: 0.9129923574368018, 14: 0.9861702127659575, 15: 0.9005376344086021, 16: 0.855646970830217, 17: 0.8291678917965304, 18: 0.9589743589743589,

# Save trained model

In [67]:
# Save a trained model
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
# output_model_file = os.path.join(PYTORCH_PRETRAINED_BERT_CACHE, "bert27catsround0.1.bin")
output_model_file = os.path.join('/data/home/gicgpu/sprint4/model/', "bert27trainontrainthenVal.bin")

torch.save(model_to_save.state_dict(), output_model_file)

# Load a trained model that you have fine-tuned
model_state_dict = torch.load(output_model_file)
model = BertForMultiLabelSequenceClassification.from_pretrained(args['bert_model'], num_labels = num_labels, state_dict=model_state_dict)
model.to(device)

04/16/2019 03:55:13 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/gicgpu/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/16/2019 03:55:13 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/gicgpu/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp7zyj80fq
04/16/2019 03:55:17 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwis

In [123]:
model

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwis

# Model Evaluation

In [ ]:
eval()


In [69]:
Path(DATA_PATH, "test.csv")

PosixPath('/data/home/gicgpu/sprint4/data/test.csv')

In [70]:
test = pd.read_csv(Path(DATA_PATH, "test.csv"))

In [74]:
test = test.drop("Index", axis=1)

In [86]:
test.to_csv(Path(DATA_PATH, "test.csv"), index=False)

In [69]:
len(test.Reference.unique())

19

In [66]:
# eval_f1()

In [39]:
def predict(model, path, test_filename='test.csv'):
    predict_processor = MultiLabelTextProcessor(path)
    test_examples = predict_processor.get_test_examples(path, test_filename, size=-1)
    
    # Hold input data for returning it 
    input_data = [{ 'Reference': input_example.guid, 'text': input_example.text_a } for input_example in test_examples]

    test_features = convert_examples_to_features(
        test_examples, label_list, args['max_seq_length'], tokenizer)
    
    logger.info("***** Running prediction *****")
    logger.info("  Num examples = %d", len(test_examples))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args['eval_batch_size'])
    
    all_logits = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
        input_ids, input_mask, segment_ids = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)
            logits = logits.sigmoid()

        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    return pd.merge(pd.DataFrame(input_data), pd.DataFrame(all_logits, columns=label_list), left_index=True, right_index=True)

In [93]:
result = predict(model, DATA_PATH)

04/17/2019 03:11:07 - INFO - root -   ***** Running prediction *****
04/17/2019 03:11:07 - INFO - root -     Num examples = 3438
04/17/2019 03:11:07 - INFO - root -     Batch size = 8


In [94]:
result

,Reference,text,(Management of a) Tax function,Anti-Tax Avoidance Directive (ATAD),Controlled Foreign Corporation,Corporate Tax,Country-by-Country Reporting,Digital Tax,Double Tax Treaty,EU Mandatory Disclosure Directive (DAC6),...,Permanent Establishment (PE),Principal purpose test,Section 892,Sovereign Immunity,Tax Governance Framework,Tax audit,Tax compliance,Tax dispute,Value-added tax (VAT),Withholding Tax
0,15,OECD/G20 Base Erosion and Profit Shifting Project,0.003044,0.000463,0.000574,0.001024,0.000703,0.002957,0.001392,0.000712,...,0.001248,0.000536,0.000602,0.000585,0.001071,0.001626,0.001138,0.001213,0.001534,0.001609
1,15,Addressing the Tax Challenges of the,0.002656,0.001092,0.000343,0.001406,0.000622,0.003077,0.002149,0.000899,...,0.001788,0.000371,0.000414,0.000652,0.001134,0.003130,0.001982,0.002956,0.001092,0.001084
2,15,Digitalisation of the Economy,0.053961,0.004193,0.004587,0.006153,0.001270,0.876471,0.007152,0.005327,...,0.034298,0.003201,0.002622,0.006448,0.012398,0.002304,0.003404,0.003071,0.085376,0.005087
3,15,Public Consultation Document,0.001372,0.001472,0.000378,0.001080,0.000919,0.002173,0.001455,0.000690,...,0.000704,0.000393,0.000497,0.000459,0.000933,0.001599,0.000968,0.000937,0.000955,0.001575
4,15,Public Consultation Document,0.001372,0.001472,0.000378,0.001080,0.000919,0.002173,0.001455,0.000690,...,0.000704,0.000393,0.000497,0.000459,0.000933,0.001599,0.000968,0.000937,0.000955,0.001575
5,15,Following a mandate by G20 Finance Ministers i...,0.008591,0.005943,0.001007,0.006257,0.001127,0.031505,0.005491,0.005138,...,0.003537,0.000819,0.000840,0.001499,0.011445,0.002681,0.002059,0.004058,0.004306,0.002282
6,15,"on BEPS, working through its Task Force on the...",0.026746,0.010515,0.004537,0.005921,0.004654,0.957679,0.022958,0.007104,...,0.034356,0.008705,0.004044,0.006191,0.012617,0.013311,0.002608,0.019606,0.170600,0.004676
7,15,"Interim Report in March 2018, Tax Challenges A...",0.003487,0.000647,0.000546,0.000813,0.000546,0.003020,0.001132,0.001162,...,0.002301,0.000729,0.000503,0.000908,0.001494,0.001823,0.001273,0.001203,0.002801,0.001008
8,15,Report 2018. One of the important conclusions ...,0.002212,0.000510,0.000543,0.000837,0.002185,0.001370,0.001818,0.001059,...,0.001090,0.000962,0.001185,0.000688,0.001101,0.003332,0.000885,0.002034,0.001625,0.002067
9,15,review the impact of digitalisation on nexus a...,0.006703,0.001405,0.001091,0.001370,0.000573,0.062245,0.002521,0.002040,...,0.001443,0.000641,0.001172,0.001601,0.004082,0.001134,0.001712,0.000965,0.004292,0.001635


cing12.html	0.060398	0.088003	0.068467	0.065018	0.045323	0.048387	0.085089	0.026707	0.047605	0.220333
1	Executive summary\n\nOn 19 October 2018, the S...	tax_controversy9.html	0.240468	0.194115	0.105509	0.113727	0.035817	0.034487	0.071500	0.028623	0.038994	0.062349
2	Companies remain under pressure to respond to ...	tax_transparency2.html	0.036964	result = predict(model, DATA_PATH)


In [95]:
result.columns

Index(['Reference', 'text', '(Management of a) Tax function',
       'Anti-Tax Avoidance Directive (ATAD)', 'Controlled Foreign Corporation',
       'Corporate Tax', 'Country-by-Country Reporting', 'Digital Tax',
       'Double Tax Treaty', 'EU Mandatory Disclosure Directive (DAC6)',
       'Economic substance', 'General anti-avoidance rule (GAAR)',
       'Goods and services tax (GST)', 'Interest deductibility', 'Local file',
       'Mandatory disclosure rules', 'Master file',
       'Multilateral Instrument (MLI)', 'Mutual agreement procedure',
       'Permanent Establishment (PE)', 'Principal purpose test', 'Section 892',
       'Sovereign Immunity', 'Tax Governance Framework', 'Tax audit',
       'Tax compliance', 'Tax dispute', 'Value-added tax (VAT)',
       'Withholding Tax'],
      dtype='object')

In [96]:
def recombine_test(dataframe):
    """
    Dataframe -> Dataframe
    
    Takes the results of a dataframe of results from a test dataset split into paragraphs
    and recombines the predictions into document-level by taking the max
    
    """
    unique_refs = dataframe.Reference.unique()
    combine_list = []
    test_labels = dataframe.columns[2:]

    for unique_ref in unique_refs:
        dict_for_ref = dict()
        dict_for_ref["Reference"] = unique_ref
        dict_for_ref["text"] = ' '.join(list(result[result["Reference"]==unique_ref]["text"]))
        for label in test_labels:
            dict_for_ref[label] = result[result["Reference"]==unique_ref][label].max()
        combine_list.append(dict_for_ref)
    return pd.DataFrame(combine_list)

In [97]:
recombined_results = recombine_test(result)

In [102]:
result

,Reference,text,(Management of a) Tax function,Anti-Tax Avoidance Directive (ATAD),Controlled Foreign Corporation,Corporate Tax,Country-by-Country Reporting,Digital Tax,Double Tax Treaty,EU Mandatory Disclosure Directive (DAC6),...,Permanent Establishment (PE),Principal purpose test,Section 892,Sovereign Immunity,Tax Governance Framework,Tax audit,Tax compliance,Tax dispute,Value-added tax (VAT),Withholding Tax
0,15,OECD/G20 Base Erosion and Profit Shifting Project,0.003044,0.000463,0.000574,0.001024,0.000703,0.002957,0.001392,0.000712,...,0.001248,0.000536,0.000602,0.000585,0.001071,0.001626,0.001138,0.001213,0.001534,0.001609
1,15,Addressing the Tax Challenges of the,0.002656,0.001092,0.000343,0.001406,0.000622,0.003077,0.002149,0.000899,...,0.001788,0.000371,0.000414,0.000652,0.001134,0.003130,0.001982,0.002956,0.001092,0.001084
2,15,Digitalisation of the Economy,0.053961,0.004193,0.004587,0.006153,0.001270,0.876471,0.007152,0.005327,...,0.034298,0.003201,0.002622,0.006448,0.012398,0.002304,0.003404,0.003071,0.085376,0.005087
3,15,Public Consultation Document,0.001372,0.001472,0.000378,0.001080,0.000919,0.002173,0.001455,0.000690,...,0.000704,0.000393,0.000497,0.000459,0.000933,0.001599,0.000968,0.000937,0.000955,0.001575
4,15,Public Consultation Document,0.001372,0.001472,0.000378,0.001080,0.000919,0.002173,0.001455,0.000690,...,0.000704,0.000393,0.000497,0.000459,0.000933,0.001599,0.000968,0.000937,0.000955,0.001575
5,15,Following a mandate by G20 Finance Ministers i...,0.008591,0.005943,0.001007,0.006257,0.001127,0.031505,0.005491,0.005138,...,0.003537,0.000819,0.000840,0.001499,0.011445,0.002681,0.002059,0.004058,0.004306,0.002282
6,15,"on BEPS, working through its Task Force on the...",0.026746,0.010515,0.004537,0.005921,0.004654,0.957679,0.022958,0.007104,...,0.034356,0.008705,0.004044,0.006191,0.012617,0.013311,0.002608,0.019606,0.170600,0.004676
7,15,"Interim Report in March 2018, Tax Challenges A...",0.003487,0.000647,0.000546,0.000813,0.000546,0.003020,0.001132,0.001162,...,0.002301,0.000729,0.000503,0.000908,0.001494,0.001823,0.001273,0.001203,0.002801,0.001008
8,15,Report 2018. One of the important conclusions ...,0.002212,0.000510,0.000543,0.000837,0.002185,0.001370,0.001818,0.001059,...,0.001090,0.000962,0.001185,0.000688,0.001101,0.003332,0.000885,0.002034,0.001625,0.002067
9,15,review the impact of digitalisation on nexus a...,0.006703,0.001405,0.001091,0.001370,0.000573,0.062245,0.002521,0.002040,...,0.001443,0.000641,0.001172,0.001601,0.004082,0.001134,0.001712,0.000965,0.004292,0.001635


In [98]:
recombined_results = recombined_results[label_list]

In [99]:
recombined_results.head()

,(Management of a) Tax function,Anti-Tax Avoidance Directive (ATAD),Controlled Foreign Corporation,Corporate Tax,Country-by-Country Reporting,Digital Tax,Double Tax Treaty,EU Mandatory Disclosure Directive (DAC6),Economic substance,General anti-avoidance rule (GAAR),...,Permanent Establishment (PE),Principal purpose test,Section 892,Sovereign Immunity,Tax Governance Framework,Tax audit,Tax compliance,Tax dispute,Value-added tax (VAT),Withholding Tax
0,0.565863,0.070124,0.050526,0.392442,0.993504,0.999539,0.999217,0.116845,0.999112,0.140802,...,0.998668,0.069137,0.607566,0.160612,0.706224,0.999475,0.136711,0.998861,0.409591,0.999835
1,0.237646,0.590536,0.295598,0.999770,0.658533,0.995187,0.334760,0.436822,0.998501,0.451808,...,0.968165,0.093563,0.070805,0.092779,0.052677,0.997626,0.285971,0.588859,0.963702,0.999671
2,0.149199,0.996306,0.318034,0.999069,0.203927,0.998625,0.274437,0.169203,0.994047,0.013196,...,0.997990,0.031619,0.955607,0.186714,0.046119,0.375709,0.999481,0.047518,0.252601,0.999754
3,0.064799,0.079855,0.046066,0.013134,0.034842,0.109568,0.396354,0.030295,0.021112,0.006137,...,0.019716,0.046870,0.016212,0.007157,0.005967,0.281438,0.011187,0.982878,0.039796,0.058485
4,0.108322,0.026580,0.062988,0.999678,0.021008,0.045231,0.024651,0.035223,0.102719,0.006786,...,0.038597,0.026678,0.083684,0.047255,0.023218,0.025705,0.016205,0.005099,0.919066,0.037344


In [100]:
len(recombined_results)

20

In [101]:
recombined_results.to_csv(Path(DATA_PATH, "bert2.1_predictions_round3.csv"), index=False)